# 2.assess-heterogeneity 

This section of the notebook uses buscar's clustering module to assess single-cell heterogeneity. We'll focus on three specific datasets: **CFReT**, **MitoCheck**, and **CPJUMP (crispir)**. The goal is to use our clustering algorithms to identify cellular heterogeneity at the single-cell level. 

A key advantage of using these datasets is that they include ground-truth labels. This allows us to evaluate whether our clustering algorithms are identifying biologically meaningful groups in a data-driven way, and to assess the accuracy of our approach.

In [1]:
import sys
import pathlib

import pickle
import polars as pl

sys.path.append("../../")
from utils.heterogeneity import assess_heterogeneity


/home/erikserrano/Programs/miniconda3/envs/buscar/lib/python3.12/site-packages/louvain/__init__.py:54: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


Setting paths

In [2]:
# set module and data directory paths
download_module_path = pathlib.Path("../0.download-data/").resolve(strict=True)
sc_profiles_path = (download_module_path / "data" / "sc-profiles").resolve(strict=True)

# set paths for the profiles
mitocheck_profile_path = (sc_profiles_path / "mitocheck" / "concat_mitocheck_cp_profiles_shared_feats.parquet").resolve(strict=True)
cfret_profile_path = (sc_profiles_path / "cfret" / "localhost230405150001_sc_feature_selected.parquet").resolve(strict=True)
cpjump1_crispir_path = (download_module_path / "results" / "concat_crispr_profiles.parquet").resolve(strict=True)

# create signature output paths
results_dir = pathlib.Path("./results").resolve()
results_dir.mkdir(exist_ok=True)

Loading datasets

In [3]:
# load all profiles
mitocheck_profile_df = pl.read_parquet(mitocheck_profile_path)
cfret_profile_df = pl.read_parquet(cfret_profile_path)
cpjump1_crispir_df = pl.read_parquet(cpjump1_crispir_path)

Splitting datasets, only selected the treated profiles

In [4]:
# separate metadata based on phenotypic class
mito_trt = mitocheck_profile_df.filter(pl.col("Mitocheck_Phenotypic_Class") != "negcon")

# split metadata and features
mito_meta = mito_trt.columns[:12]
mito_features = mito_trt.columns[12:]

In [5]:
# only selected treatment profiles from cfret
cfret_trt = cfret_profile_df.filter(pl.col("Metadata_treatment") != "DMSO")

# split metadata and features for cfret
cfret_meta = cfret_trt.columns[:19]
cfret_feats = cfret_trt.columns[19:]

In [6]:
# selecting only treated profiles from cpjump1
cpjump1_trt = cpjump1_crispir_df.filter(pl.col("Metadata_pert_type") == "trt")

# split metadata and features for cpjump1
cpjump1_meta = cpjump1_trt.columns[:18]
cpjump1_feats = cpjump1_trt.columns[18:]


## Clustering profiles

In [7]:
mitocheck_cluster_results = assess_heterogeneity(profiles=mito_trt, meta=mito_meta, features=mito_features, n_trials=500, n_jobs=1, study_name="mitocheck_heterogeneity", seed=0)
with open(results_dir / "mitocheck_cluster_results.pkl", "wb") as f:
    pickle.dump(mitocheck_cluster_results, f)

[I 2025-08-07 15:05:10,296] A new study created in memory with name: mitocheck_heterogeneity


  0%|          | 0/500 [00:00<?, ?it/s]

/home/erikserrano/Development/BuSCar/notebooks/1.compound-prioritization/../../utils/heterogeneity.py:113: UserWarning: Ignoring svd_solver='auto' and using randomized, sklearn.decomposition._truncated_svd.TruncatedSVD only supports dict_keys(['arpack', 'randomized']).
  sc.pp.pca(


[I 2025-08-07 15:05:16,867] Trial 0 finished with value: 0.022089131118488235 and parameters: {'n_neighbors': 77, 'dim_reduction': 'pca', 'pca_solver': 'auto', 'pca_components': 23, 'resolution': 1.4636269269064541, 'method': 'leiden', 'dist_metric': 'cosine', 'louv_clustering_imp': 'vtraag', 'leid_clustering_imp': 'igraph'}. Best is trial 0 with value: 0.022089131118488235.


/home/erikserrano/Development/BuSCar/notebooks/1.compound-prioritization/../../utils/heterogeneity.py:215: FutureWarning: In the future, the default backend for leiden will be igraph instead of leidenalg.

 To achieve the future defaults please pass: flavor="igraph" and n_iterations=2.  directed must also be False to work with igraph's implementation.
  sc.tl.leiden(


...

[I 2025-08-07 15:10:41,958] Trial 156 finished with value: 0.17049007392862134 and parameters: {'n_neighbors': 152, 'dim_reduction': 'pca', 'pca_solver': 'auto', 'pca_components': 47, 'resolution': 0.14094101324564784, 'method': 'leiden', 'dist_metric': 'euclidean', 'louv_clustering_imp': 'vtraag', 'leid_clustering_imp': 'leidenalg'}. Best is trial 151 with value: 0.22510736528853403.


/home/erikserrano/Development/BuSCar/notebooks/1.compound-prioritization/../../utils/heterogeneity.py:113: UserWarning: Ignoring svd_solver='auto' and using randomized, sklearn.decomposition._truncated_svd.TruncatedSVD only supports dict_keys(['arpack', 'randomized']).
  sc.pp.pca(


[I 2025-08-07 15:10:48,298] Trial 157 finished with value: 0.15860726303386716 and parameters: {'n_neighbors': 150, 'dim_reduction': 'pca', 'pca_solver': 'auto', 'pca_components': 48, 'resolution': 0.15518455074893475, 'method': 'leiden', 'dist_metric': 'euclidean', 'louv_clustering_imp': 'igraph', 'leid_clustering_imp': 'leidenalg'}. Best is trial 151 with value: 0.22510736528853403.


/home/erikserrano/Development/BuSCar/notebooks/1.compound-prioritization/../../utils/heterogeneity.py:113: UserWarning: Ignoring svd_solver='auto' and using randomized, sklearn.decomposition._truncated_svd.TruncatedSVD only supports dict_keys(['arpack', 'randomized']).
  sc.pp.pca(


[I 2025-08-07 15:10:53,979] Trial 158 finished with value: 0.16420258373433705 and parameters: {'n_neighbors': 151, 'dim_reduction': 'pca', 'pca_solver': 'auto', 'pca_components': 49, 'resolution': 0.13200787359160093, 'method': 'leiden', 'dist_metric': 'euclidean', 'louv_clustering_imp': 'igraph', 'leid_clustering_imp': 'leidenalg'}. Best is trial 151 with value: 0.22510736528853403.
[I 2025-08-07 15:10:59,712] Trial 159 finished with value: 0.15860726303386716 and parameters: {'n_neighbors': 150, 'dim_reduction': 'pca', 'pca_solver': 'arpack', 'pca_components': 48, 'resolution': 0.1414251376061341, 'method': 'leiden', 'dist_metric': 'euclidean', 'louv_clustering_imp': 'igraph', 'leid_clustering_imp': 'leidenalg'}. Best is trial 151 with value: 0.22510736528853403.
[I 2025-08-07 15:11:05,928] Trial 160 finished with value: 0.16439189413381966 and parameters: {'n_neighbors': 150, 'dim_reduction': 'pca', 'pca_solver': 'arpack', 'pca_components': 49, 'resolution': 0.1490742715134526, 'me

In [8]:
cfret_cluster_results = assess_heterogeneity(profiles=cfret_trt, meta=cfret_meta, features=cfret_feats, n_trials=500, n_jobs=1, study_name="cfret_heterogeneity", seed=0)
with open(results_dir / "cfret_cluster_results.pkl", "wb") as f:
    pickle.dump(cfret_cluster_results, f)

[I 2025-08-07 15:46:44,690] A new study created in memory with name: cfret_heterogeneity


  0%|          | 0/500 [00:00<?, ?it/s]

`resolution` parameter has no effect for flavor "igraph"


[I 2025-08-07 15:47:39,058] Trial 0 finished with value: 0.035042695598890734 and parameters: {'n_neighbors': 188, 'dim_reduction': 'raw', 'pca_solver': 'randomized', 'pca_components': 95, 'resolution': 1.0463606417159637, 'method': 'louvain', 'dist_metric': 'cosine', 'louv_clustering_imp': 'igraph', 'leid_clustering_imp': 'igraph'}. Best is trial 0 with value: 0.035042695598890734.


/home/erikserrano/Development/BuSCar/notebooks/1.compound-prioritization/../../utils/heterogeneity.py:113: UserWarning: Ignoring svd_solver='auto' and using randomized, sklearn.decomposition._truncated_svd.TruncatedSVD only supports dict_keys(['arpack', 'randomized']).
  sc.pp.pca(
`resolution` parameter has no effect for flavor "igraph"


...

[I 2025-08-07 19:20:28,245] Trial 357 finished with value: 0.1286328522035976 and parameters: {'n_neighbors': 194, 'dim_reduction': 'pca', 'pca_solver': 'auto', 'pca_components': 49, 'resolution': 0.18334899994923928, 'method': 'louvain', 'dist_metric': 'cosine', 'louv_clustering_imp': 'vtraag', 'leid_clustering_imp': 'leidenalg'}. Best is trial 279 with value: 0.12964544250258045.


/home/erikserrano/Development/BuSCar/notebooks/1.compound-prioritization/../../utils/heterogeneity.py:113: UserWarning: Ignoring svd_solver='auto' and using randomized, sklearn.decomposition._truncated_svd.TruncatedSVD only supports dict_keys(['arpack', 'randomized']).
  sc.pp.pca(


[I 2025-08-07 19:20:57,003] Trial 358 finished with value: 0.12840800791049956 and parameters: {'n_neighbors': 197, 'dim_reduction': 'pca', 'pca_solver': 'auto', 'pca_components': 42, 'resolution': 0.224333858439818, 'method': 'louvain', 'dist_metric': 'cosine', 'louv_clustering_imp': 'vtraag', 'leid_clustering_imp': 'leidenalg'}. Best is trial 279 with value: 0.12964544250258045.
[I 2025-08-07 19:21:22,965] Trial 359 finished with value: -1.0 and parameters: {'n_neighbors': 200, 'dim_reduction': 'pca', 'pca_solver': 'randomized', 'pca_components': 44, 'resolution': 0.13914542223541693, 'method': 'louvain', 'dist_metric': 'cosine', 'louv_clustering_imp': 'vtraag', 'leid_clustering_imp': 'leidenalg'}. Best is trial 279 with value: 0.12964544250258045.
[I 2025-08-07 19:22:01,221] Trial 360 finished with value: 0.053525517552143805 and parameters: {'n_neighbors': 193, 'dim_reduction': 'raw', 'pca_solver': 'auto', 'pca_components': 47, 'resolution': 0.27455773315569465, 'method': 'louvain'

/home/erikserrano/Development/BuSCar/notebooks/1.compound-prioritization/../../utils/heterogeneity.py:113: UserWarning: Ignoring svd_solver='auto' and using randomized, sklearn.decomposition._truncated_svd.TruncatedSVD only supports dict_keys(['arpack', 'randomized']).
  sc.pp.pca(


[I 2025-08-07 19:22:28,448] Trial 361 finished with value: 0.12931013004077827 and parameters: {'n_neighbors': 196, 'dim_reduction': 'pca', 'pca_solver': 'auto', 'pca_components': 51, 'resolution': 0.18626000580913712, 'method': 'louvain', 'dist_metric': 'cosine', 'louv_clustering_imp': 'vtraag', 'leid_clustering_imp': 'leidenalg'}. Best is trial 279 with value: 0.12964544250258045.
[I 2025-08-07 19:23:07,618] Trial 362 finished with value: 0.13107500232590247 and parameters: {'n_neighbors': 190, 'dim_reduction': 'raw', 'pca_solver': 'auto', 'pca_components': 41, 'resolution': 0.22999119874134563, 'method': 'louvain', 'dist_metric': 'cosine', 'louv_clustering_imp': 'vtraag', 'leid_clustering_imp': 'leidenalg'}. Best is trial 362 with value: 0.13107500232590247.
[I 2025-08-07 19:23:46,386] Trial 363 finished with value: 0.1310525927355763 and parameters: {'n_neighbors': 190, 'dim_reduction': 'raw', 'pca_solver': 'auto', 'pca_components': 41, 'resolution': 0.23884595568714925, 'method': 

In [ ]:
cpjump1_cluster_results = assess_heterogeneity(profiles=cpjump1_trt, meta=cpjump1_meta, features=cpjump1_feats, n_trials=500, n_jobs=1, study_name="cpjump1_heterogeneity", seed=0)
with open(results_dir / "cpjump1_cluster_results.pkl", "wb") as f:
    pickle.dump(cpjump1_cluster_results, f)


[I 2025-08-07 20:50:43,078] A new study created in memory with name: cpjump1_heterogeneity


  0%|          | 0/500 [00:00<?, ?it/s]